In [38]:
import geopandas as gpd
import os
import pandas as pd
from tqdm.notebook import tqdm
from PIL import Image

In [5]:
classify_dict = pd.read_csv('./class_dict.csv')
classify_dict.head(5)


,name,r,g,b
0,Animal,64,128,64
1,Archway,192,0,128
2,Bicyclist,0,128,192
3,Bridge,0,128,64
4,Building,128,0,0


In [55]:
# reading img into numpy array, color order: (B,G,R)

def countColor(fp:str):
    # counting color in img
    # input: img file path
    # output: dict {(r,g,b): int}
    img = Image.open(fp)
    count = img.getcolors()
    count = {x[1]:x[0] for x in count}
    return count 

def counterToDataframe(counter:dict, SEG_ID:int, pointId:str):
    result = pd.DataFrame({"SEG_ID":[SEG_ID],"pointId":[pointId]})
    for i, row in classify_dict.iterrows():
        bgr = (row.b,row.g,row.r)
        className = row["name"]
        if bgr in counter.keys():
            count = counter[bgr]
        else: count = 0
        result[className] = count
    return result



In [57]:
inputImgPath = "segnet-output/"
inputImgFiles = os.listdir(inputImgPath)
classCountDf = None
for i, file in tqdm(enumerate(inputImgFiles),total=len(inputImgFiles)):
    fp = inputImgPath + file
    index = file.split(".")[0]
    segId,imgId = index.split('-')
    segId = int(segId)
    colorCount = countColor(fp)
    imgCountDf = counterToDataframe(colorCount,segId,imgId)
    if classCountDf is None: 
        classCountDf = imgCountDf
    else:
        classCountDf = pd.concat([classCountDf, imgCountDf], axis=0)
    if i % 100 == 0:
        classCountDf.to_csv('./segnetClassCount.csv')

  0%|          | 0/12317 [00:00<?, ?it/s]

In [59]:
classCountDf.to_csv('./segnetClassCount.csv')
classCountDf

,SEG_ID,pointId,Animal,Archway,Bicyclist,Bridge,Building,Car,CartLuggagePram,Child,...,SUVPickupTruck,TrafficCone,TrafficLight,Train,Tree,Truck_Bus,Tunnel,VegetationMisc,Void,Wall
0,641548,010394,4,991,0,0,0,0,0,2763,...,3,0,2,0,0,1525,0,0,95643,0
0,640416,001313,1,65,0,0,0,0,6,8303,...,0,0,1237,0,0,1631,0,0,66082,0
0,620130,007230,1,672,0,0,0,0,0,2222,...,59,0,846,0,0,2195,0,0,136329,0
0,401340,009336,0,365,0,0,0,0,0,3830,...,0,0,744,0,0,7148,0,0,84051,0
0,761434,006817,8,1890,0,0,0,0,0,5028,...,1,0,89,0,0,4110,0,0,124267,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,762729,009793,0,418,0,0,0,0,0,4305,...,0,0,43,0,0,5455,0,0,82498,0
0,420928,009032,22,4208,0,0,0,0,0,4588,...,1,0,2412,0,0,2732,0,0,120717,0
0,641818,003436,30,2733,0,0,0,0,0,1073,...,0,0,1045,0,0,142,0,0,65817,0
0,820541,010376,0,1843,0,0,0,0,27,141,...,0,0,0,0,0,9,0,0,32781,0


In [61]:
streetPoints = gpd.read_file("../streetViewPoints.geojson")
streetPoints = streetPoints.to_crs(4326)

In [62]:
areaDf = pd.merge(streetPoints,classCountDf,on=["SEG_ID","pointId"])
areaDf